In [ ]:
#default_exp showdoc

In [ ]:
# export
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from IPython.display import Markdown,display
from IPython.core import page
from nbconvert import HTMLExporter

# Show doc

> Functions to show the doc cells in notebooks

All the automatic documentation of functions and classes are generated with the `show_doc` function. It displays the name, arguments, docstring along with a link to the source code on GitHub.

## Gather the information

The inspect module lets us know quickly if an object is a function or a class but it doesn't distinguish classes and enums.

In [ ]:
# export
def is_enum(cls):
    "Check if `cls` is an enum or another type of class"
    return type(cls) in (enum.Enum, enum.EnumMeta)

In [ ]:
e = enum.Enum('e', 'a b')
assert is_enum(e)
assert not is_enum(int)

### Links to documentaion

In [ ]:
#hide
#TODO: come back to this later
## export
#def _get_pytorch_index():
#    if not (Path(_file_).parent/'index_pytorch.txt').exists(): return {}
#    return json.load(open(Path(_file_).parent/'index_pytorch.txt', 'r'))
#
#def add_pytorch_index(func_name, url):
#    "Add `func_name` in the PyTorch index for automatic links."
#    index = _get_pytorch_index()
#    if not url.startswith("https://pytorch.org/docs/stable/"):
#        url = "https://pytorch.org/docs/stable/" + url
#    index[func_name] = url
#   json.dump(index, open(Path(_file_).parent/'index_pytorch.txt', 'w'), indent=2)

In [ ]:
#hide
#ind,ind_bak = Path(_file_).parent/'index_pytorch.txt',Path(_file_).parent/'index_pytorch.bak'
#if ind.exists(): shutil.move(ind, ind_bak)
#assert _get_pytorch_index() == {}
#add_pytorch_index('Tensor', 'tensors.html#torch-tensor')
#assert _get_pytorch_index() == {'Tensor':'https://pytorch.org/docs/stable/tensors.html#torch-tensor'}
#if ind_bak.exists(): shutil.move(ind_bak, ind)

In [ ]:
#hide
#add_pytorch_index('Tensor', 'tensors.html#torch-tensor')
#add_pytorch_index('device', 'tensor_attributes.html#torch-device')
#add_pytorch_index('DataLoader', 'data.html#torch.utils.data.DataLoader')

In [ ]:
# export
def is_lib_module(name):
    "Test if `name` is a library module."
    try:
        _ = importlib.import_module(f'{Config().lib_name}.{name}')
        return True
    except: return False

In [ ]:
assert is_lib_module('export')
assert not is_lib_module('transform')

In [ ]:
# export
def doc_link(name, include_bt:bool=True):
    "Create link to documentation for `name`."
    cname = f'`{name}`' if include_bt else name
    #Link to modules
    if is_lib_module(name): return f"[{cname}](/{'_'.join(name.split('.'))}.html)"
    #Link to local functions
    try_local = source_nb(name, is_name=True)
    if try_local:
        page = '.'.join(try_local.split('_')[1:]).replace('.ipynb', '.html')
        return f'[{cname}](/{page}#{name})'
    ##Link to PyTorch
    #try_pytorch = _get_pytorch_index().get(name, None)
    #if try_pytorch: return f'[{cname}]({try_pytorch})'
    #Leave as is
    return cname

This function will generate link for a module (pointing to the html conversion of the notebook that created it) and functions (pointing to the hmtl conversion of the notebook they were defined, with the first anchor found before). If the function/module is not part of the library you are writing, it will just return the name between backsticks.

In [ ]:
test_eq(doc_link('export'), f'[`export`](/export.html)')
test_eq(doc_link('DocsTestClass'), f'[`DocsTestClass`](/export.html#DocsTestClass)')
test_eq(doc_link('DocsTestClass.test'), f'[`DocsTestClass.test`](/export.html#DocsTestClass.test)')
test_eq(doc_link('Tenso'),'`Tenso`')

In [ ]:
#export
_re_backticks = re.compile(r"""
# Catches any link of the form \[`obj`\](old_link) or just `obj`,
#   to either update old links or add the link to the docs of obj
\[`      #     Opening [ and `
([^`]*)  #     Catching group with anything but a `
`\]      #     ` then closing ]
(?:      #     Beginning of non-catching group
\(       #       Opening (
[^)]*    #       Anything but a closing )
\)       #       Closing )
)        #     End of non-catching group
|        # OR
`        #     Opening `
([^`]*)  #       Anything but a `
`        #     Closing `
""", re.VERBOSE)

In [ ]:
# export
def add_doc_links(text):
    "Search for doc links for any item between backticks in `text` and isnter them"
    def _replace_link(m): return doc_link(m.group(1) or m.group(2))
    return _re_backticks.sub(_replace_link, text)

This function not only add links to backstick keywords, it also update the links that are already in the text (in case they have changed).

In [ ]:
tst = add_doc_links('This is an example of `DocsTestClass`')
test_eq(tst, "This is an example of [`DocsTestClass`](/export.html#DocsTestClass)")
tst = add_doc_links('This is an example of [`DocsTestClass`](/old_link.html)')
test_eq(tst, "This is an example of [`DocsTestClass`](/export.html#DocsTestClass)")

### Links to source

In [ ]:
#export
def _is_type_dispatch(x): return type(x).__name__ == "TypeDispatch"
def _unwrapped_type_dispatch_func(x): return x.first() if _is_type_dispatch(x) else x

def _is_property(x): return type(x)==property
def _has_property_getter(x): return _is_property(x) and hasattr(x, 'fget') and hasattr(x.fget, 'func')
def _property_getter(x): return x.fget.func if _has_property_getter(x) else x

def _unwrapped_func(x):
    x = _unwrapped_type_dispatch_func(x)
    x = _property_getter(x)
    return x

In [ ]:
#export
def get_source_link(func):
    "Return link to `func` in source code"
    func = _unwrapped_func(func)
    try: line = inspect.getsourcelines(func)[1]
    except Exception: return ''
    module = inspect.getmodule(func).__name__.replace('.', '/') + '.py'
    return f"{Config().git_url}{module}#L{line}"

Be sure to properly set the `git_url` in setting.ini (derived from `lib_name` and `branch` on top of the prefix you will need to adapt) so that those links are correct.

In [ ]:
#hide
assert get_source_link(DocsTestClass.test).startswith(Config().git_url + 'nbdev/export.py')

As important as the source code, we want to quickly jump to where the function is defined when we are in a development notebook.

In [ ]:
#export
_re_header = re.compile(r"""
# Catches any header in markdown with the title in group 1
^\s*  # Beginning of text followed by any number of whitespace
\#+   # One # or more
\s*   # Any number of whitespace
(.*)  # Catching group with anything
$     # End of text
""", re.VERBOSE)

In [ ]:
#export
NB_SOURCE_URL = Config().git_url.replace('github.com', 'nbviewer.jupyter.org/github')+ Config().nbs_path.name+'/'

def get_nb_source_link(func, local=False, is_name=None):
    "Return a link to the notebook where `func` is defined."
    func = _unwrapped_type_dispatch_func(func)
    pref = '' if local else NB_SOURCE_URL
    is_name = is_name or isinstance(func, str)
    src = source_nb(func, is_name=is_name, return_all=True)
    if src is None: return '' if is_name else get_source_link(func)
    find_name,nb_name = src
    nb = read_nb(nb_name)
    pat = re.compile(f'^{find_name}\s+=|^(def|class)\s+{find_name}\s*\(', re.MULTILINE)
    if len(find_name.split('.')) == 2:
        clas,func = find_name.split('.')
        pat2 = re.compile(f'@patch\s*\ndef\s+{func}\s*\([^:]*:\s*{clas}\s*(?:,|\))')
    else: pat2 = None
    for i,cell in enumerate(nb['cells']):
        if cell['cell_type'] == 'code':
            if re.search(pat, cell['source']):  break
            if pat2 is not None and re.search(pat2, cell['source']): break
    if re.search(pat, cell['source']) is None and (pat2 is not None and re.search(pat2, cell['source']) is None):
        return '' if is_name else get_function_source(func)
    header_pat = re.compile(r'^\s*#+\s*(.*)$')
    while i >= 0:
        cell = nb['cells'][i]
        if cell['cell_type'] == 'markdown' and _re_header.search(cell['source']):
            title = _re_header.search(cell['source']).groups()[0]
            anchor = '-'.join([s for s in title.split(' ') if len(s) > 0])
            return f'{pref}{nb_name}#{anchor}' 
        i-=1
    return f'{pref}{nb_name}'

In [ ]:
test_eq(get_nb_source_link(DocsTestClass.test), get_nb_source_link(DocsTestClass))
test_eq(get_nb_source_link('DocsTestClass'), get_nb_source_link(DocsTestClass))
test_eq(get_nb_source_link(check_re), f'{NB_SOURCE_URL}00_export.ipynb#Finding-patterns')
test_eq(get_nb_source_link(check_re, local=True), f'00_export.ipynb#Finding-patterns')

You can either pass an object or its name (by default `is_name` will look if `func` is a string or not to decide if it's `True` or `False`, but you can override if there is some inconsistent behavior). `local` will return a local link, otherwise it will point to a the notebook on github wrapped in [nbviewer](https://nbviewer.jupyter.org/).

In [ ]:
# export
def nb_source_link(func, is_name=None, disp=True):
    "Show a relative link to the notebook where `func` is defined"
    is_name = is_name or isinstance(func, str)
    func_name = func if is_name else qual_name(func)
    link = get_nb_source_link(func, local=True, is_name=is_name)
    if disp: display(Markdown(f'[{func_name}]({link})'))
    else: return link

This function assumes you are in one notebook in the development folder, otherwise you can use `disp=False` to get the relative link. You can either pass an object or its name (by default `is_name` will look if `func` is a string or not to decide if it's `True` or `False`, but you can override if there is some inconsistent behavior).

In [ ]:
nb_source_link(check_re)

[check_re](00_export.ipynb#Finding-patterns)

In [ ]:
test_eq(nb_source_link(check_re, disp=False), f'00_export.ipynb#Finding-patterns')
test_eq(nb_source_link('check_re', disp=False), f'00_export.ipynb#Finding-patterns')

## Show documentation

In [ ]:
# export
from fastscript import Param

In [ ]:
# export
def type_repr(t):
    "Representation of type `t` (in a type annotation)"
    if (isinstance(t, Param)): return f'"{t.help}"'
    if getattr(t, '__args__', None):
        args = t.__args__
        if len(args)==2 and args[1] == type(None):
            return f'`Optional`\[{type_repr(args[0])}\]'
        reprs = ', '.join([type_repr(o) for o in args])
        return f'{doc_link(get_name(t))}\[{reprs}\]'
    else: return doc_link(get_name(t))

The representation tries to find doc links if possible.

In [ ]:
tst = type_repr(Optional[DocsTestClass])
test_eq(tst, '`Optional`\\[[`DocsTestClass`](/export.html#DocsTestClass)\\]')
tst = type_repr(Union[int, float])
test_eq(tst, '`Union`\\[`int`, `float`\\]')
test_eq(type_repr(Param("description")), '"description"')

In [ ]:
# export
_arg_prefixes = {inspect._VAR_POSITIONAL: '\*', inspect._VAR_KEYWORD:'\*\*'}

def format_param(p):
    "Formats function param to `param:Type=val` with font weights: param=bold, val=italic"
    arg_prefix = _arg_prefixes.get(p.kind, '') # asterisk prefix for *args and **kwargs
    res = f"**{arg_prefix}`{p.name}`**"
    if hasattr(p, 'annotation') and p.annotation != p.empty: res += f':{type_repr(p.annotation)}'
    if p.default != p.empty:
        default = getattr(p.default, 'func', p.default) #For partials
        default = getattr(default, '__name__', default) #Tries to find a name
        if is_enum(default.__class__):                  #Enum have a crappy repr
            res += f'=*`{default.__class__.__name__}.{default.name}`*'
        else: res += f'=*`{repr(default)}`*'
    return res

In [ ]:
sig = inspect.signature(notebook2script)
params = [format_param(p) for _,p in sig.parameters.items()]
test_eq(params, ['**`fname`**=*`None`*', '**`silent`**=*`False`*', '**`to_dict`**=*`False`*'])

In [ ]:
# export
def _format_enum_doc(enum, full_name):
    "Formatted `enum` definition to show in documentation"
    vals = ', '.join(enum.__members__.keys())
    return f'<code>{full_name}</code>',f'<code>Enum</code> = [{vals}]'

In [ ]:
#hide
tst =  _format_enum_doc(e, 'e')
test_eq(tst, ('<code>e</code>', '<code>Enum</code> = [a, b]'))

In [ ]:
# export
def _escape_chars(s):
    return s.replace('_', '\_')

def _format_func_doc(func, full_name=None):
    "Formatted `func` definition to show in documentation"
    try:
        sig = inspect.signature(func)
        fmt_params = [format_param(param) for name,param
                  in sig.parameters.items() if name not in ('self','cls')]
    except: fmt_params = []
    name = f'<code>{full_name or func.__name__}</code>'
    arg_str = f"({', '.join(fmt_params)})"
    f_name = f"<code>class</code> {name}" if inspect.isclass(func) else name
    return f'{f_name}',f'{name}{arg_str}'

In [ ]:
#hide
test_eq(_format_func_doc(notebook2script), ('<code>notebook2script</code>', 
   '<code>notebook2script</code>(**`fname`**=*`None`*, **`silent`**=*`False`*, **`to_dict`**=*`False`*)'))

In [ ]:
# export
def _format_cls_doc(cls, full_name):
    "Formatted `cls` definition to show in documentation"
    parent_class = inspect.getclasstree([cls])[-1][0][1][0]
    name,args = _format_func_doc(cls, full_name)
    if parent_class != object: args += f' :: {doc_link(get_name(parent_class))}'
    return name,args

In [ ]:
#hide
test_eq(_format_cls_doc(DocsTestClass, 'DocsTestClass'), ('<code>class</code> <code>DocsTestClass</code>', 
                                                          '<code>DocsTestClass</code>()'))

In [ ]:
# export
def show_doc(elt, doc_string=True, name=None, title_level=None, disp=True, default_cls_level=2):
    "Show documentation for element `elt`. Supported types: class, function, and enum."
    elt = getattr(elt, '__func__', elt)
    qname = name or qual_name(elt)
    if inspect.isclass(elt):
        if is_enum(elt.__class__):   name,args = _format_enum_doc(elt, qname)
        else:                        name,args = _format_cls_doc (elt, qname)
    elif callable(elt):  name,args = _format_func_doc(elt, qname)
    else:                            name,args = f"<code>{qname}</code>", ''
    link = get_source_link(elt)
    source_link = f'<a href="{link}" class="source_link" style="float:right">[source]</a>'
    title_level = title_level or (default_cls_level if inspect.isclass(elt) else 4)
    doc =  f'<h{title_level} id="{qname}" class="doc_header">{name}{source_link}</h{title_level}>'
    doc += f'\n\n> {args}\n\n' if len(args) > 0 else '\n\n'
    if doc_string and inspect.getdoc(elt): doc += add_doc_links(inspect.getdoc(elt))
    if disp: display(Markdown(doc))
    else: return doc

`doc_string` determines if we show the docstring of the function or not. `name` can be used to provide an alternative to the name automatically found. `title_level` determines the level of the anchor (default 3 for classes and 4 for functions). If `disp` is `False`, the function returns the markdown code instead of displaying it.

For instance

```python
show_doc(notebook2script)
```
will display
<h4 id="notebook2script" class="doc_header"><code>notebook2script</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/export.py#L277" class="source_link" style="float:right">[source]</a></h4>

> <code>notebook2script</code>(**`fname`**=*`None`*, **`silent`**=*`False`*, **`to_pkl`**=*`False`*)

Convert `fname` or all the notebook satisfying `all_fs`.

### Integration test -

In [ ]:
#hide
show_doc(DocsTestClass)

<h2 id="DocsTestClass" class="doc_header"><code>class</code> <code>DocsTestClass</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/export.py#L309" class="source_link" style="float:right">[source]</a></h2>

> <code>DocsTestClass</code>()



In [ ]:
#hide
show_doc(DocsTestClass.test)

<h4 id="DocsTestClass.test" class="doc_header"><code>DocsTestClass.test</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/export.py#L310" class="source_link" style="float:right">[source]</a></h4>

> <code>DocsTestClass.test</code>()



In [ ]:
#hide
show_doc(notebook2script)

<h4 id="notebook2script" class="doc_header"><code>notebook2script</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/export.py#L293" class="source_link" style="float:right">[source]</a></h4>

> <code>notebook2script</code>(**`fname`**=*`None`*, **`silent`**=*`False`*, **`to_dict`**=*`False`*)

Convert notebooks matching `fname` to modules

In [ ]:
#hide
show_doc(check_re)

<h4 id="check_re" class="doc_header"><code>check_re</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/export.py#L17" class="source_link" style="float:right">[source]</a></h4>

> <code>check_re</code>(**`cell`**, **`pat`**, **`code_only`**=*`True`*)

Check if `cell` contains a line with regex `pat`

### The doc command

In [ ]:
#export
def md2html(md):
    "Convert markdown `md` to HTML code"
    if nbconvert.__version__ < '5.5.0': return HTMLExporter().markdown2html(md)
    else: return HTMLExporter().markdown2html(defaultdict(lambda: defaultdict(dict)), md)

In [ ]:
#export
def doc(elt):
    "Show `show_doc` info in preview window when used in a notebook"
    md = show_doc(elt, disp=False)
    output = md2html(md)
    if IN_COLAB: get_ipython().run_cell_magic(u'html', u'', output)
    else:
        try: page.page({'text/html': output})
        except: display(Markdown(md))

## Export -

In [ ]:
#hide
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_cli.ipynb.
Converted index.ipynb.
